In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# Chapitre 7 — Simulations

Ce chapitre traite de **simulations** stochastiques (aléatoires) :

- laver des voitures (car-wash) ;
- guichets et files d’attente ;
- débit de bières dans un bistrot ;
- circulation de caddies au supermarché ;
- etc.

Les exercices facultatifs peuvent être résolus :

- soit avec les **structures de données** du chapitre POO (par ex. files, files de priorité),
- soit avec des **listes Python** classiques, sans POO.

Ce notebook se concentre sur :

- la **structure** typique d’une simulation ;
- l’utilisation d’une **boucle de temps** (par ex. une itération par seconde) ;
- l’utilisation du module `random` ;
- une petite section d’exercices supplémentaires avec un **grader**.

## 7.1 Structure générale d’une simulation discrète

Les simulations du chapitre suivent souvent ce schéma :

1. Choisir une **durée totale** (par ex. une journée de 10h → `36000` secondes) ;
2. Initialiser les **variables d’état** : file d’attente, temps de service restant, compteurs, ... ;
3. Pour chaque **unité de temps** (souvent 1 seconde) :
   - générer des **événements aléatoires** (arrivées, commandes, etc.) ;
   - mettre à jour l’état (service, attente, départs, ...) ;
   - accumuler des statistiques ;
4. Après la simulation, calculer des **moyennes** (temps d’attente, pourcentages, ...).

On utilise souvent le module `random` :

- arrivées en moyenne toutes les `x` secondes → on peut modéliser par :
  - soit une probabilité `p` à chaque seconde ;
  - soit un temps aléatoire entre arrivées (variable géométrique / exponentielle discrète).

In [ ]:
# Exemple minimal : simulation d'arrivées simples

import random

def simulate_arrivals(duration, mean_interval):
    """
    Simule des arrivées pendant 'duration' secondes.
    'mean_interval' est le temps moyen entre deux arrivées (en secondes).
    Ici, on modélise grossièrement via une probabilité p = 1/mean_interval à chaque seconde.
    Renvoie le nombre d'arrivées.
    """
    p = 1.0 / mean_interval
    arrivals = 0
    for t in range(duration):
        if random.random() < p:
            arrivals += 1
    return arrivals

print("Arrivées sur 3600 s (1h), intervalle moyen 240 s :", simulate_arrivals(3600, 240))

## 7.2 Exercice 7.1 — Car-wash (concept)

On simule une **station de lavage** avec une file d’attente :

- une nouvelle voiture arrive en moyenne toutes les `x` secondes (ex : 240 s) ;
- un lavage dure `time_wash` (ex : 5 minutes = 300 s) ;
- une journée dure `day_duration` (ex : 10h = 36000 s) ;
- on simule plusieurs journées (ex : 10) ;
- on s’intéresse au **temps moyen d’attente** dans la file.

Classe suggérée pour la machine (issue du texte) :

```python
class CarWash:
    def __init__(self, tw):
        self.time_wash = tw
        self.time_remaining = 0

    def busy(self):
        return self.time_remaining > 0

    def tick(self):
        if self.busy():
            self.time_remaining -= 1

    def start_wash(self):
        self.time_remaining = self.time_wash

    def __str__(self):
        s = f"lavage, total = {self.time_wash},"
        return s + f" reste = {self.time_remaining}"
```

Le programme principal maintient :

- une **file** d’attente (par ex. une liste des instants d’arrivée) ;
- un objet `CarWash` ;
- des compteurs pour le **temps total d’attente**, nombre de voitures lavées, non lavées, etc.

## 7.3 Exercice 7.2 — Simulation d’un guichet (concept)

On simule un **guichet** avec :

- arrivées en moyenne toutes les 40 secondes ;
- une **patience** par client (entre 2 et 10 minutes) ;
- un temps de service entre 30 secondes et 5 minutes ;
- une journée de 8h (28800 s) ;
- plusieurs journées (ex : 10).

On mesure :

- nombre de clients servis ;
- nombre de clients partis par impatience (file trop lente) ;
- nombre de clients encore en attente à la fermeture ;
- temps moyen d’attente.

UML de la classe `Desk` suggérée :

```text
Desk
time_busy: int
__init__()
__str__(): str
is_busy(): bool
tick()
new_customer()
```

Les exercices 7.3–7.5 étendent ces idées à plusieurs guichets, bistrot, supermarché, etc.

# 7.X Exercices supplémentaires d'entraînement (avec grader)

Pour ce chapitre, le grader ne testera pas des simulations complètes (trop longues),
mais des **briques de base** utiles dans ces simulations :

- génération d’arrivées aléatoires ;
- mise à jour d’une file d’attente ;
- estimation de temps d’attente moyens.

Le fichier **`grader_chapitre_7.py`** testera les fonctions :

- `next_arrival_time` ;
- `simulate_single_server_queue` ;
- `average_waiting_time`.

## Exercice S1 — Temps d’arrivée suivant (~exponentiel discret)

**Temps conseillé : 10 à 15 minutes**

Écris une fonction :

```python
def next_arrival_time(current_time, mean_interval):
    ...
```

qui renvoie un **instant d’arrivée futur** (entier) en fonction :

- de l’instant actuel `current_time` (entier, en secondes) ;
- d’un intervalle moyen `mean_interval` (en secondes, `> 0`).

Modèle simplifié :

- génère un délai aléatoire `delta` entier ≥ 0 ;
- tel que la **valeur moyenne** de `delta` soit proche de `mean_interval` ;
- renvoie `current_time + delta`.

Par simplicité, tu peux modéliser `delta` comme un multiple de `mean_interval` ou utiliser une distribution géométrique
approximée : par exemple, additionner à `delta` des pas de 1 jusqu'à obtenir `random.random() < 1/mean_interval`.

Le grader vérifiera seulement :

- que la valeur retournée est `>= current_time` ;
- et que les valeurs sont globalement raisonnables (pas d’erreur, pas de valeurs négatives).

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- une approche simple :
  ```python
  delta = 0
  p = 1.0 / mean_interval
  while random.random() >= p:
      delta += 1
  return current_time + delta
  ```
- cette boucle simule un nombre de pas jusqu’au "succès" avec probabilité `p`.

</details>

In [ ]:
# Exercice S1 (Chapitre 7) — next_arrival_time

import random

def next_arrival_time(current_time, mean_interval):
    """Renvoie un instant d'arrivée futur (entier >= current_time).

    Modèle simplifié: délai aléatoire avec moyenne ~ mean_interval (en secondes).
    """
    # TODO: à implémenter par toi
    # ... ton code ici ...
    raise NotImplementedError("À compléter")

In [ ]:
# === Vérification de l'exercice S1 ===
import sys
import os

GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"
BOLD = "\033[1m"

try:
    grader_path = 'grader_chapitre_7.py'
    
    import importlib.util
    spec = importlib.util.spec_from_file_location("grader", grader_path)
    grader = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(grader)
    
    import types
    temp_module = types.ModuleType("temp")
    
    for name in dir():
        if not name.startswith('_') and name not in ['grader', 'spec', 'temp_module', 'grader_path']:
            try:
                temp_module.__dict__[name] = eval(name)
            except:
                pass
    
    print(f"{BOLD}=== Évaluation de l'exercice S1 ==={RESET}")
    
    grade_func = getattr(grader, f'grade_s1')
    result = grade_func(temp_module)
    
    if result:
        print(f"{GREEN}SUCCÈS: Tous les tests sont passés!{RESET}")
    else:
        print(f"{RED}ÉCHEC: Certains tests n'ont pas passé{RESET}")
        
except Exception as e:
    print(f"{RED}ERREUR lors de la vérification: {e}{RESET}")
    import traceback
    traceback.print_exc()


## Exercice S2 — Simulation simplifiée d’une file M/M/1 discrète

**Temps conseillé : 30 à 40 minutes**

Écris une fonction :

```python
def simulate_single_server_queue(duration, mean_interarrival, service_time):
    ...
```

qui simule **une file d’attente avec un seul serveur** pendant `duration` secondes.

Paramètres :

- `duration` (int) : durée totale de la simulation en secondes ;
- `mean_interarrival` (float) : intervalle moyen entre arrivées (en secondes) ;
- `service_time` (int) : temps de service **constant** pour chaque client (en secondes).

Hypothèses simplifiées :

- les arrivées sont générées avec `next_arrival_time` ;
- la file est **FIFO** (First-In, First-Out) ;
- le service est non préemptif (on finit un client avant d’en prendre un autre) ;
- on ne modélise pas la patience (personne ne part).

La fonction doit renvoyer un tuple :

```python
(n_served, waiting_times)
```

où :

- `n_served` est le nombre de clients **complètement servis** dans la période ;
- `waiting_times` est une liste des temps d’attente (en secondes) de chaque client servi.

Indications pour la structure :

- maintenir une file (liste) des instants d’arrivée des clients en attente ;
- maintenir `next_arrival` : instant d’arrivée du prochain client ;
- maintenir `server_busy_until` : instant où le serveur sera libre ;
- itérer `t` de `0` à `duration-1` en gérant les arrivées et les départs.

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- initialise `queue = []`, `waiting_times = []`, `n_served = 0` ;
- fixe `next_arrival = next_arrival_time(0, mean_interarrival)` et `server_busy_until = 0` ;
- à chaque seconde `t` :
  - si `t == next_arrival`, ajoute `t` à `queue` et calcule le prochain `next_arrival` ;
  - si `t >= server_busy_until` et `queue` non vide, démarre un service :
    * `arrival_time = queue.pop(0)` (FIFO),
    * le temps d’attente est `t - arrival_time` → ajoute à `waiting_times`,
    * mets `server_busy_until = t + service_time`,
    * incrémente `n_served` ;
- renvoie `n_served, waiting_times`.

</details>

In [ ]:
# Exercice S2 (Chapitre 7) — simulate_single_server_queue

def simulate_single_server_queue(duration, mean_interarrival, service_time):
    """
    Simule une file d'attente avec un seul serveur pendant 'duration' secondes.

    Retourne (n_served, waiting_times).
    """
    # TODO: à implémenter par toi
    # - utiliser next_arrival_time pour les arrivées
    # - gérer une file FIFO d'instants d'arrivée
    # - lancer un service si le serveur est libre et la file non vide
    # - accumuler les temps d'attente dans waiting_times
    # ... ton code ici ...
    raise NotImplementedError("À compléter")

In [ ]:
# === Vérification de l'exercice S2 ===
import sys
import os

GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"
BOLD = "\033[1m"

try:
    grader_path = 'grader_chapitre_7.py'
    
    import importlib.util
    spec = importlib.util.spec_from_file_location("grader", grader_path)
    grader = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(grader)
    
    import types
    temp_module = types.ModuleType("temp")
    
    for name in dir():
        if not name.startswith('_') and name not in ['grader', 'spec', 'temp_module', 'grader_path']:
            try:
                temp_module.__dict__[name] = eval(name)
            except:
                pass
    
    print(f"{BOLD}=== Évaluation de l'exercice S2 ==={RESET}")
    
    grade_func = getattr(grader, f'grade_s2')
    result = grade_func(temp_module)
    
    if result:
        print(f"{GREEN}SUCCÈS: Tous les tests sont passés!{RESET}")
    else:
        print(f"{RED}ÉCHEC: Certains tests n'ont pas passé{RESET}")
        
except Exception as e:
    print(f"{RED}ERREUR lors de la vérification: {e}{RESET}")
    import traceback
    traceback.print_exc()


## Exercice S3 — Temps d’attente moyen sur plusieurs simulations

**Temps conseillé : 10 à 15 minutes**

Écris une fonction :

```python
def average_waiting_time(n_runs, duration, mean_interarrival, service_time):
    ...
```

qui :

1. exécute `n_runs` fois `simulate_single_server_queue` ;
2. collecte tous les temps d’attente de tous les clients servis ;
3. renvoie le **temps d’attente moyen** (float).

Si aucun client n’est servi dans toutes les simulations (cas limite), tu peux renvoyer `0.0`.

<details>
<summary><strong>Aide (dévoiler si besoin)</strong></summary>

- crée une liste `all_waits = []` ;
- pour `i` de 0 à `n_runs-1`, appelle `simulate_single_server_queue` et étends `all_waits` avec les `waiting_times` ;
- si `all_waits` est vide, renvoie `0.0` ;
- sinon, renvoie `sum(all_waits) / len(all_waits)`.

</details>

In [ ]:
# Exercice S3 (Chapitre 7) — average_waiting_time

def average_waiting_time(n_runs, duration, mean_interarrival, service_time):
    """
    Renvoie le temps d'attente moyen (en secondes) sur n_runs simulations.
    """
    # TODO: à implémenter par toi
    # - appeler simulate_single_server_queue plusieurs fois
    # - accumuler tous les waiting_times
    # - renvoyer la moyenne
    # ... ton code ici ...
    raise NotImplementedError("À compléter")

In [ ]:
# === Vérification de l'exercice S3 ===
import sys
import os

GREEN = "\033[92m"
RED = "\033[91m"
RESET = "\033[0m"
BOLD = "\033[1m"

try:
    grader_path = 'grader_chapitre_7.py'
    
    import importlib.util
    spec = importlib.util.spec_from_file_location("grader", grader_path)
    grader = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(grader)
    
    import types
    temp_module = types.ModuleType("temp")
    
    for name in dir():
        if not name.startswith('_') and name not in ['grader', 'spec', 'temp_module', 'grader_path']:
            try:
                temp_module.__dict__[name] = eval(name)
            except:
                pass
    
    print(f"{BOLD}=== Évaluation de l'exercice S3 ==={RESET}")
    
    grade_func = getattr(grader, f'grade_s3')
    result = grade_func(temp_module)
    
    if result:
        print(f"{GREEN}SUCCÈS: Tous les tests sont passés!{RESET}")
    else:
        print(f"{RED}ÉCHEC: Certains tests n'ont pas passé{RESET}")
        
except Exception as e:
    print(f"{RED}ERREUR lors de la vérification: {e}{RESET}")
    import traceback
    traceback.print_exc()


## Comment utiliser le grader externe du chapitre 7

1. Assure‑toi d'avoir complété :
   - `next_arrival_time`,
   - `simulate_single_server_queue`,
   - `average_waiting_time`.

2. Sauvegarde ce notebook (`chapitre_7_interactif.ipynb`).

3. Dans un terminal, exécute le fichier **`grader_chapitre_7.py`** (placé à côté de ce notebook) :

```bash
python grader_chapitre_7.py
```

Le grader :

- importera ce notebook comme un module Python ;
- exécutera une série de tests cachés ;
- affichera pour chaque exercice un statut : `S1`, `S2`, `S3` → `Réussi` ou `Échoué`.

Tu sauras donc si ta logique de base pour les simulations est correcte, **sans voir la solution**.

In [ ]:
# Lancer le grader du chapitre 7 directement depuis ce notebook

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, importlib.util

BASE = "/content/drive/MyDrive/1ereB_info"
os.chdir(BASE)
print("Répertoire courant:", os.getcwd())

spec = importlib.util.spec_from_file_location(
    "grader_chapitre_7",
    os.path.join(BASE, "grader_chapitre_7.py"),
)
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
mod.main()
